# Валидация дат и временных рядов

В этом ноутбуке мы рассмотрим валидацию дат и временных рядов с использованием различных ограничений.

## Подготовка окружения

In [ ]:
import sys
sys.path.append('../../')

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from core.data.schema import DataSchema, ColumnSchema, DataType
from core.data.validation import SchemaValidator

## Создание схемы с временными ограничениями

In [ ]:
# Создание схемы данных с временными ограничениями
schema = DataSchema([
    ColumnSchema(
        name='id',
        data_type=DataType.INTEGER,
        required=True,
        constraints={
            'unique': True
        }
    ),
    ColumnSchema(
        name='created_at',
        data_type=DataType.DATETIME,
        required=True,
        constraints={
            'min': '2020-01-01',
            'max': '2023-12-31',
            'format': '%Y-%m-%d %H:%M:%S'
        }
    ),
    ColumnSchema(
        name='updated_at',
        data_type=DataType.DATETIME,
        required=True,
        constraints={
            'min': '2020-01-01',
            'max': '2023-12-31',
            'format': '%Y-%m-%d %H:%M:%S',
            'after': 'created_at'  # Пользовательское ограничение
        }
    ),
    ColumnSchema(
        name='value',
        data_type=DataType.FLOAT,
        required=True,
        constraints={
            'min': 0,
            'max': 1000
        }
    )
])

## Создание валидатора с временными проверками

In [ ]:
class DateTimeSchemaValidator(SchemaValidator):
    def _validate_datetime(self, series: pd.Series, schema: ColumnSchema) -> None:
        super()._validate_datetime(series, schema)
        
        # Проверка на формат даты
        format_str = schema.constraints.get('format')
        if format_str:
            try:
                pd.to_datetime(series, format=format_str)
            except ValueError as e:
                raise ValueError(f"Колонка {schema.name} должна соответствовать формату {format_str}")
        
        # Проверка на минимальную дату
        min_date = schema.constraints.get('min')
        if min_date:
            min_date = pd.to_datetime(min_date)
            if (series < min_date).any():
                raise ValueError(f"Колонка {schema.name} должна содержать даты не ранее {min_date}")
        
        # Проверка на максимальную дату
        max_date = schema.constraints.get('max')
        if max_date:
            max_date = pd.to_datetime(max_date)
            if (series > max_date).any():
                raise ValueError(f"Колонка {schema.name} должна содержать даты не позднее {max_date}")
        
        # Проверка на последовательность дат
        if schema.constraints.get('after'):
            after_column = schema.constraints['after']
            if after_column not in self.data.columns:
                raise ValueError(f"Колонка {after_column} не найдена в схеме")
            
            after_series = self.data[after_column]
            if (series <= after_series).any():
                raise ValueError(f"Колонка {schema.name} должна содержать даты позже, чем в колонке {after_column}")

## Подготовка тестовых данных

In [ ]:
# Создание корректных данных
valid_data = pd.DataFrame({
    'id': [1, 2, 3],
    'created_at': [
        '2023-01-01 10:00:00',
        '2023-01-02 11:00:00',
        '2023-01-03 12:00:00'
    ],
    'updated_at': [
        '2023-01-01 11:00:00',
        '2023-01-02 12:00:00',
        '2023-01-03 13:00:00'
    ],
    'value': [100, 200, 300]
})

# Создание некорректных данных
invalid_data = pd.DataFrame({
    'id': [1, 2, 3],
    'created_at': [
        '2019-01-01 10:00:00',  # Дата до минимальной
        '2023-01-02 11:00:00',
        '2024-01-03 12:00:00'   # Дата после максимальной
    ],
    'updated_at': [
        '2023-01-01 09:00:00',  # Дата до created_at
        '2023-01-02 12:00:00',
        '2023-01-03 13:00:00'
    ],
    'value': [100, 200, 300]
})

## Создание и использование валидатора

In [ ]:
# Создание валидатора
validator = DateTimeSchemaValidator(schema)

# Валидация корректных данных
try:
    validator.validate(valid_data)
    print("Данные успешно валидированы")
except ValueError as e:
    print(f"Ошибка валидации: {e}")

# Валидация некорректных данных
try:
    validator.validate(invalid_data)
except ValueError as e:
    print(f"Ошибка валидации: {e}")

## Валидация временных рядов

In [ ]:
# Создание временного ряда
dates = pd.date_range(start='2023-01-01', end='2023-01-10', freq='D')
values = np.random.normal(100, 10, len(dates))

time_series = pd.DataFrame({
    'date': dates,
    'value': values
})

# Проверка на пропущенные даты
date_diffs = time_series['date'].diff()
if date_diffs.nunique() > 1:
    print("Временной ряд содержит пропущенные даты")
else:
    print("Временной ряд непрерывный")

# Проверка на выбросы
z_scores = np.abs((time_series['value'] - time_series['value'].mean()) / time_series['value'].std())
outliers = time_series[z_scores > 3]
if not outliers.empty:
    print("Обнаружены выбросы:")
    print(outliers)
else:
    print("Выбросов не обнаружено")